In [1]:
import numpy as np
import random
from tqdm import tqdm
from numba import jit

# Assume your previous functions: check_winner, play_game, sim_games
@jit(nopython=True)
def check_winner(board, x, y, player, rows, columns, inarow):
    # Directions: vertical, horizontal, diagonal /, diagonal \
    directions = [(1, 0), (0, 1), (1, 1), (1, -1)]
    for dx, dy in directions:
        count = 1
        for d in [1, -1]:
            step = 1
            while True:
                nx, ny = x + step * dx * d, y + step * dy * d
                if 0 <= nx < rows and 0 <= ny < columns and board[nx, ny] == player:
                    count += 1
                    step += 1
                    if count >= inarow:
                        return True
                else:
                    break
        # Reset count for next direction
    return False



import time

# Assume your previous functions: check_winner, play_game, sim_games


def play_game_two_bots(board, bot1_agent, bot2_agent, player, rows, columns, inarow):
    """
    Simulates a game between two bots using their agent functions.
    """
    current_player = player
    bot1_move_times = []
    bot2_move_times = []

    while True:
        possible_moves = np.where(board[0, :] == 0)[0]
        if possible_moves.size == 0:
            return 0, bot1_move_times, bot2_move_times  # Draw game

        observation = {'board': board.flatten(), 'mark': current_player}
        configuration = {'rows': rows, 'columns': columns, 'inarow': inarow}

        if current_player == 1:
            start_time = time.time()
            move = bot1_agent(observation, configuration)
            end_time = time.time()
            bot1_move_times.append(end_time - start_time)
        else:
            start_time = time.time()
            move = bot2_agent(observation, configuration)
            end_time = time.time()
            bot2_move_times.append(end_time - start_time)

        rows_available = np.where(board[:, move] == 0)[0]
        if rows_available.size == 0:
            continue  # Invalid move, try again
        row = rows_available[-1]
        board[row, move] = current_player

        if check_winner(board, row, move, current_player, rows, columns, inarow):
            return current_player, bot1_move_times, bot2_move_times  # Return the winner and move times

        current_player = 3 - current_player  # Switch player


def simulate_multiple_games(bot1_agent, bot2_agent, num_games, rows, columns, inarow):
    """
    Simulate a number of games between two agents and collect statistics.
    """
    bot1_wins = 0
    bot2_wins = 0
    draws = 0
    bot1_average_move_times = []
    bot2_average_move_times = []

    # Wrap the loop with tqdm to display the progress bar
    for game in tqdm(range(num_games), desc="Simulating games"):
        board = np.zeros((rows, columns), dtype=int)
        starting_player = random.choice([1, 2])  # Randomize starting player

        result, bot1_move_times, bot2_move_times = play_game_two_bots(
            board, bot1_agent, bot2_agent, starting_player, rows, columns, inarow)

        if result == 1:
            bot1_wins += 1
        elif result == 2:
            bot2_wins += 1
        else:
            draws += 1

        # Calculate average time per move for first 10 moves
        bot1_first_10_moves = bot1_move_times[:10]
        bot2_first_10_moves = bot2_move_times[:10]
        if bot1_first_10_moves:
            avg_bot1_time = sum(bot1_first_10_moves) / len(bot1_first_10_moves)
            bot1_average_move_times.append(avg_bot1_time)
        if bot2_first_10_moves:
            avg_bot2_time = sum(bot2_first_10_moves) / len(bot2_first_10_moves)
            bot2_average_move_times.append(avg_bot2_time)

    # Calculate average of these averages
    overall_avg_bot1_time = sum(bot1_average_move_times) / len(bot1_average_move_times) if bot1_average_move_times else 0
    overall_avg_bot2_time = sum(bot2_average_move_times) / len(bot2_average_move_times) if bot2_average_move_times else 0

    # Print final statistics
    print(f"Total games: {num_games}")
    print(f"Bot1 wins: {bot1_wins} ({(bot1_wins / num_games) * 100:.2f}%)")
    print(f"Bot2 wins: {bot2_wins} ({(bot2_wins / num_games) * 100:.2f}%)")
    print(f"Draws: {draws} ({(draws / num_games) * 100:.2f}%)")

    # Print average move time (only counting first 10 moves per game)
    print(f"Average time per move (first 10 moves per game) - Bot1: {overall_avg_bot1_time:.5f} seconds")
    print(f"Average time per move (first 10 moves per game) - Bot2: {overall_avg_bot2_time:.5f} seconds")


# Example usage:
#simulate_multiple_games(agent_bot1, agent_bot2, num_games=100, rows=6, columns=7, inarow=4)


In [2]:
import importlib
import agent_exhaust_monty2
import agent_exhaust_monty

# Reload the modules to refresh their content
importlib.reload(agent_exhaust_monty2)
importlib.reload(agent_exhaust_monty)

# Import the agents with the original import style
from agent_exhaust_monty2 import agent as agent
from agent_monty2 import agent as agent1


In [3]:
# Define both agents using the same agent function or different ones if desired
def agent_bot1(observation, configuration):
    # This is where you define the first bot's strategy (using the agent function)
    return agent(observation, configuration)  # Calls the previously defined `agent` function

def agent_bot2(observation, configuration):
    # This is where you define the second bot's strategy (could be different from agent_bot1)
    return agent1(observation, configuration)  # Calls the same agent function, can be customized

In [4]:
simulate_multiple_games(agent_bot1, agent_bot2, num_games=1, rows=6, columns=7, inarow=4)

Simulating games: 100%|██████████| 1/1 [00:30<00:00, 30.22s/it]

Total games: 1
Bot1 wins: 0 (0.00%)
Bot2 wins: 1 (100.00%)
Draws: 0 (0.00%)
Average time per move (first 10 moves per game) - Bot1: 1.42505 seconds
Average time per move (first 10 moves per game) - Bot2: 0.26661 seconds
